In [1]:
## Imports
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
from sklearn.utils import class_weight
from collections import Counter
import torch
import numpy as np
import evaluate

## Load the dataset

In [17]:


# === Tokenizer ===
tokenizer = AutoTokenizer.from_pretrained("roberta-base")



In [48]:

# 1) Load the single-file CSV
raw_ds = load_dataset("csv", data_files="./AI_Human.csv")["train"]

# 2) Split 80 % / 10 % / 10 %
tmp        = raw_ds.train_test_split(test_size=0.2, seed=42)
train_ds   = tmp["train"]
tmp2       = tmp["test"].train_test_split(test_size=0.5, seed=42)
val_ds     = tmp2["train"]
test_ds    = tmp2["test"]


def tokenize_fn(batch):
    enc = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=256)
    # Force to int for HF Trainer and PyTorch loss:
    enc["labels"] = [int(x) for x in batch["generated"]]
    return enc

token_train = train_ds.map(tokenize_fn, batched=True)
token_val   = val_ds.map(tokenize_fn,   batched=True)
token_test  = test_ds.map(tokenize_fn,  batched=True)

for ds in (token_train, token_val, token_test):
    ds.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/389788 [00:00<?, ? examples/s]

Map:   0%|          | 0/48723 [00:00<?, ? examples/s]

Map:   0%|          | 0/48724 [00:00<?, ? examples/s]

In [50]:
print(token_train[0]["labels"], type(token_train[0]["labels"]))


tensor(0) <class 'torch.Tensor'>


In [ ]:
import numpy as np, torch, collections

labels_train   = train_ds["generated"]               
weights_np     = class_weight.compute_class_weight(
                    "balanced",
                    classes=np.unique(labels_train),
                    y=labels_train,
                 )
class_weights  = torch.tensor(weights_np, dtype=torch.float)
print("Train distr:", collections.Counter(labels_train))
print("Class wts   :", class_weights)

Train distr: Counter({0.0: 244783, 1.0: 145005})
Class wts   : tensor([0.7962, 1.3441])


In [67]:
import torch.nn.functional as F
class WeightedTrainer(Trainer):
    def __init__(self, class_weights, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = F.cross_entropy(
            logits, labels,
            weight=self.class_weights.to(logits.device)
        )
        return (loss, outputs) if return_outputs else loss




In [53]:
# === Evaluation metrics ===
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.argmax(torch.tensor(logits), dim=1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1": f1.compute(predictions=preds, references=labels)["f1"]
    }


In [63]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2)


In [64]:
def hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0.0, 0.3),
        "lora_dropout": trial.suggest_float("lora_dropout", 0.0, 0.3),
        "per_device_train_batch_size": 8,
        "num_train_epochs": 1,
    }

In [65]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="no",
    learning_rate=2e-5,  # gets overridden by Optuna
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    load_best_model_at_end=False,
    report_to="none",
    fp16=True,
)

In [68]:
trainer = WeightedTrainer(                      # ✏️ CHANGE
    class_weights=class_weights,                # 🔧 NEW
    model_init=model_init,
    args=training_args,
    train_dataset=token_train,                  # ✏️ CHANGE
    eval_dataset=token_val,                     # ✏️ CHANGE
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

best_run = trainer.hyperparameter_search(
    direction="maximize",
    n_trials=10,
    hp_space=hp_space,
    backend="optuna"
)

print("Best hyperparameters:", best_run.hyperparameters)

C:\Users\Dayso\AppData\Local\Temp\ipykernel_6420\4118468332.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[I 2025-08-01 00:20:32,210] A new study created in memory with name: no-name-2103c78a-d66d-499e-84cd-2356ca70cff9
Trying to set lora_dropout in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.003700,0.001693,0.999631,0.999504


[I 2025-08-01 01:40:09,325] Trial 0 finished with value: 1.9991349153198215 and parameters: {'learning_rate': 1.06616603285535e-05, 'weight_decay': 0.23300051397406885, 'lora_dropout': 0.16514461747589287}. Best is trial 0 with value: 1.9991349153198215.
Trying to set lora_dropout in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.000000,0.000928,0.999836,0.999780


[I 2025-08-01 03:03:23,254] Trial 1 finished with value: 1.999615457244391 and parameters: {'learning_rate': 1.6035947409854068e-05, 'weight_decay': 0.18655360030594448, 'lora_dropout': 0.16533904614382647}. Best is trial 1 with value: 1.999615457244391.
Trying to set lora_dropout in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.000300,0.008037,0.998830,0.998431


[I 2025-08-01 21:48:05,601] Trial 8 pruned. 
Trying to set lora_dropout in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.000000,0.004770,0.999159,0.998872


[I 2025-08-01 23:23:40,088] Trial 9 pruned. 


Best hyperparameters: {'learning_rate': 1.6035947409854068e-05, 'weight_decay': 0.18655360030594448, 'lora_dropout': 0.16533904614382647}


In [ ]:
print(token_train[0]["labels"], type(token_train[0]["labels"]), token_train[0]["labels"].shape)

tensor(0.) <class 'torch.Tensor'> torch.Size([])


In [59]:
print(token_train[0]["labels"], type(token_train[0]["labels"]))  # Should be int (not float), e.g. tensor(0)
print(token_train[:8]["labels"])  # Should be a 1D tensor/list, e.g. tensor([0, 1, 0, ...])
print(token_train[:8]["labels"].shape)  # Should be torch.Size([8])


tensor(0) <class 'torch.Tensor'>
tensor([0, 1, 0, 0, 0, 0, 0, 1])
torch.Size([8])
